In [1]:
%matplotlib inline
import plotly.graph_objects as go 
from datetime import datetime as dt
import dash_bootstrap_components as dbc
import numpy as np
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from pmdarima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
import matplotlib
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import plotly.tools
from io import BytesIO
import base64
import json
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)



#Loading Data

df = pd.read_csv("Python Sensor Data - Project.csv")

df["From"]= pd.to_datetime(df["From"])
df["To"] = pd.to_datetime(df["To"])
df



,Machine,Monitor,From,To,Total Acceleration Min ((m/s²)²),Total Acceleration Max ((m/s²)²),Total Acceleration Avg ((m/s²)²),Axial Velocity Min (mm/s),Axial Velocity Max (mm/s),Axial Velocity Avg (mm/s),...,Horizontal Velocity Avg (mm/s),Vertical Velocity Min (mm/s),Vertical Velocity Max (mm/s),Vertical Velocity Avg (mm/s),Temperature Min (°C),Temperature Max (°C),Temperature Avg (°C),Audio Min (dB),Audio Max (dB),Audio Avg (dB)
0,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-16 01:30:00,2020-12-16 02:00:00,112.30,120.00,116.14,0.97,1.40,1.08,...,0.77,0.31,0.48,0.35,24.84,24.86,24.84,113.30,114.50,113.91
1,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-16 02:00:00,2020-12-16 02:30:00,111.20,153.80,118.32,0.94,1.89,1.14,...,0.85,0.31,0.85,0.40,24.71,24.98,24.77,112.10,117.60,113.49
2,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-16 02:30:00,2020-12-16 03:00:00,111.40,131.70,118.00,0.95,2.24,1.14,...,0.86,0.31,0.70,0.42,24.35,24.84,24.62,112.10,117.10,113.47
3,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-16 03:00:00,2020-12-16 03:30:00,111.60,148.90,118.71,1.03,1.31,1.14,...,0.87,0.44,0.57,0.50,24.31,25.11,24.64,112.10,115.70,113.39
4,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-16 03:30:00,2020-12-16 04:00:00,103.20,149.30,117.39,0.32,1.32,1.05,...,0.82,0.38,0.61,0.48,25.10,25.90,25.46,77.04,120.50,111.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-30 10:00:00,2020-12-30 10:30:00,105.50,106.10,105.81,0.32,0.43,0.36,...,0.32,0.41,0.52,0.45,18.83,22.26,21.00,92.18,94.19,93.31
638,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-30 10:30:00,2020-12-30 11:00:00,101.10,212.12,107.77,0.00,1.80,0.63,...,0.61,0.00,0.96,0.58,24.97,28.86,26.04,59.60,125.90,72.96
639,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-30 11:00:00,2020-12-30 11:30:00,101.02,196.90,114.81,0.00,2.34,0.69,...,0.70,0.00,0.97,0.60,23.29,26.73,25.71,58.75,126.99,82.79
640,Y7 Grinding Spindle Bearing housing,Y7 Grinding Spindle Bearing housing,2020-12-30 11:30:00,2020-12-30 12:00:00,101.81,216.02,125.59,0.00,1.63,0.75,...,0.78,0.00,0.96,0.61,24.09,28.58,26.19,58.37,125.84,85.93


In [3]:
#Initiating a global DF to be used for various callbacks
globalVib = pd.DataFrame()

In [8]:
#Initiating a Dictionary with Vibration Paramaters and Value Index
Dict = {"MinAcceleration":["Total Acceleration Min ((m/s²)²)", "Min Acceleration Index"],
        "AvgAcceleration":["Total Acceleration Avg ((m/s²)²)", "Avg Acceleration Index"],
        "MaxAcceleration":["Total Acceleration Max ((m/s²)²)", "Max Acceleration Index"],
        "MinAxial":["Axial Velocity Min (mm/s)", "Min Axial Velocity Index"],
        "MaxAxial":["Axial Velocity Max (mm/s)", "Max Axial Velocity Index"],
        "AvgAxial":["Axial Velocity Avg (mm/s)", "Avg Axial Velocity Index"],
        "MinHorizontal":["Horizontal Velocity Min (mm/s)", "Min Horizontal Velocity Index"],
        "MaxHorizontal":["Horizontal Velocity Max (mm/s)", "Max Horizontal Velocity Index"],
        "AvgHorizontal":["Horizontal Velocity Avg (mm/s)", "Avg Horizontal Velocity Index"],
        "MinVertical":["Vertical Velocity Min (mm/s)", "Min Vertical Velocity Index"],
        "MaxVertical":["Vertical Velocity Max (mm/s)", "Max Vertical Velocity Index"],
        "AvgVertical":["Vertical Velocity Avg (mm/s)", "Avg Vertical Velocity Index"],
        "MinTemperature":["Temperature Min (°C)", "Min Temperature Index"],
        "MaxTemperature":["Temperature Max (°C)", "Max Temperature Index"],
        "AvgTemperature":["Temperature Avg (°C)", "Avg Temperature Index"],
        "MinAudio":["Audio Min (dB)","Min Audio Index"],
        "MaxAudio":["Audio Max (dB)","Max Audio Index"],
        "AvgAudio":["Audio Avg (dB)","Avg Audio Index"] }

#Style Dictionaries for Tabs
tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

#Initiating app 
app = JupyterDash(__name__,suppress_callback_exceptions=True,external_stylesheets=[dbc.themes.DARKLY])

#Layout for Analysis Tab
analysis_layout = html.Div([  

    #Creating Input fields to take Thresholds for different parameters
    dbc.Row([
        dbc.Col(children = [html.H6("Accel. Alert threshold (m/s²)²",className='bg-warning font-italic font-weight-bold'),
        dcc.Input(id='Acceleration Alert Threshold',type='number',placeholder='Insert Acceleration Alert Threshold',debounce=True, min=50, max=250, step=5,disabled=False,readOnly=False,required=False,size="20",value=105,className='mb-4')
                           ]),
        dbc.Col(children = [html.H6("Accel. Danger threshold (m/s²)²",className='bg-danger text-white'),    
        dcc.Input(id='Acceleration Danger Threshold',type='number',placeholder='Insert Acceleration Danger Threshold',debounce=True, min=50, max=250, step=5,disabled=False,readOnly=False,required=False,size="20",value=125)
                           ]),
        dbc.Col(children = [html.H6("Velocity Alert threshold (mm/s)",className='bg-warning text-white'),     
        dcc.Input(id='Velocity Alert Threshold',type='number',placeholder='Insert Velocity Alert Threshold',debounce=True, min=0, max=25, step=0.5,disabled=False,readOnly=False,required=False,size="20",value=1)
                           ]),
        dbc.Col(children = [html.H6("Velocity Danger threshold (mm/s)",className='bg-danger text-white'),      
        dcc.Input(id='Velocity Danger Threshold',type='number',placeholder='Insert Velocity Danger Threshold',debounce=True, min=0, max=25, step=0.5,disabled=False,readOnly=False,required=False,size="20",value=2)
                           ]),
        dbc.Col(children = [html.H6("Temp. Alert threshold (°C)",className='bg-warning text-white'),      
        dcc.Input(id='Temperature Alert Threshold',type='number',placeholder='Insert Temperature Alert Threshold',debounce=True, min=0, max=100, step=1,disabled=False,readOnly=False,required=False,size="20",value=25)
                           ]),
        dbc.Col(children = [html.H6("Temp. Danger Threshold (°C)",className='bg-danger text-white'),      
        dcc.Input(id='Temperature Danger Threshold',type='number',placeholder='Insert Temperature Danger Threshold',debounce=True, min=0, max=100, step=1,disabled=False,readOnly=False,required=False,size="20",value=30)
                           ]),
        dbc.Col(children = [html.H6("Audio Alert Threshold (dB)",className='bg-warning text-white'),     
        dcc.Input(id='Audio Alert Threshold',type='number',placeholder='Insert Audio Alert Threshold',debounce=True, min=0, max=200, step=2,disabled=False,readOnly=False,required=False,size="20",value=100)
                           ]),
        dbc.Col(children = [html.H6("Audio Danger Threshold (dB)",className='bg-danger text-white'),     
        dcc.Input(id='Audio Danger Threshold',type='number',placeholder='Insert Audio Danger Threshold',debounce=True, min=0, max=200, step=2,disabled=False,readOnly=False,required=False,size="20",value=110)
                           ]) 
    ]),
    
    #Creating dropdown with options that lets the user to select a particular vibration parameter. 
    #Value of options will be referenced from the dictionary Dict
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='parameter_dropdown', 
            optionHeight = 25, searchable=True,placeholder = "Please Select", clearable = True,
            multi = False, style={"width": "65%","color": "#222A2A" },
            value="AvgAcceleration", options=[
                {"label":"Min Total Acceleration","value":"MinAcceleration"},
                    {"label":"Max Total Acceleration","value":"MaxAcceleration"},
                    {"label":"Avg Total Acceleration","value":"AvgAcceleration"},
                    {"label":"Min Axial Velocity","value":"MinAxial"},
                    {"label":"Max Axial Velocity","value":"MaxAxial"},
                    {"label":"Avg Axial Velocity","value":"AvgAxial"},
                    {"label":"Min Horizontal Velocity","value":"MinHorizontal"},
                    {"label":"Max Horizontal Velocity","value":"MaxHorizontal"},
                    {"label":"Avg Horizontal Velocity","value":"AvgHorizontal"},
                    {"label":"Min Vertical Velocity","value":"MinVertical"},
                    {"label":"Max Vertical Velocity","value":"MaxVertical"},
                    {"label":"Avg Vertical Velocity","value":"AvgVertical"},
                    {"label":"Min Temperature","value":"MinTemperature"},
                    {"label":"Max Temperature","value":"MaxTemperature"},
                    {"label":"Avg Temperature","value":"AvgTemperature"},
                    {"label":"Min Audio","value":"MinAudio"},
                    {"label":"Max Audio","value":"MaxAudio"},
                    {"label":"Avg Audio","value":"AvgAudio"}
                
            ])
        
        ),
        
        #Creating a DatePicker that lets User select a specific from and To date. Other Analysis will depend on the Date range selected
       dbc.Col(dcc.DatePickerRange(
        id="DatePicker",
        calendar_orientation="horizontal",
        day_size=39,
        start_date_placeholder_text="Start Date", 
        end_date_placeholder_text="End Date",  
        with_portal=False,  
        first_day_of_week=0,  
        reopen_calendar_on_clear=True,
        is_RTL=False,  
        clearable=True,  
        number_of_months_shown=1,  
        min_date_allowed=dt(2020, 1, 1),  # minimum date allowed on the DatePickerRange component
        max_date_allowed=dt(2022, 1, 1),  # maximum date allowed on the DatePickerRange component
        initial_visible_month=dt(2020, 12, 1),  
        start_date=dt(2020, 12, 1).date(),
        end_date=dt(2021, 1, 5).date(),
        display_format='MM/DD/YYYY',  
        month_format='MMMM, YYYY',  
        minimum_nights=2,  
        persistence=True,
        persisted_props=['start_date'],
        persistence_type='session',  
        updatemode='singledate',
        style={'width': '200%', 'height': '90%'},
        className='mb-4'        
    )
       
       )]),
    
    #Creating Graph components that will be displayed based on callback. 
    dbc.Row([
       dbc.Col(dcc.Graph(id='Piegraph'), width = {'size':5}),
       dbc.Col(dcc.Graph(id='lineGraph'))
       
    ])
])


#Define Callback for Analysis Tab
@app.callback(
    [Output('Piegraph', 'figure'), Output('lineGraph','figure')],
    [
     Input('DatePicker','start_date'),
     Input('DatePicker','end_date'),
     Input('parameter_dropdown', 'value'),
     Input('Acceleration Alert Threshold', 'value'),
     Input('Acceleration Danger Threshold', 'value'),
     Input('Velocity Alert Threshold', 'value'),
     Input('Velocity Danger Threshold', 'value'),
     Input('Temperature Alert Threshold', 'value'),
     Input('Temperature Danger Threshold', 'value'),
     Input('Audio Alert Threshold', 'value'),
     Input('Audio Danger Threshold', 'value')  
    ]
)

#Defining Function for Analysis Tab
def new_graph(start_date,end_date,parameter_dropdown,a,b,c,d,e,f,g,h):
    
    VibrationDF = df.copy() #Copying to a new DF to create bins based on input threshold
    
#Create New Column Names that holds the Condition of Asset based on the bins assigned
       
    VibrationDF["Min Acceleration Index"] = pd.cut(df["Total Acceleration Min ((m/s²)²)"], bins=[0,a,b,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Max Acceleration Index"] = pd.cut(df["Total Acceleration Max ((m/s²)²)"], bins=[0,a,b,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Avg Acceleration Index"] = pd.cut(df["Total Acceleration Avg ((m/s²)²)"], bins=[0,a,b,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Min Axial Velocity Index"] = pd.cut(df["Axial Velocity Min (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Max Axial Velocity Index"] = pd.cut(df["Axial Velocity Max (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Avg Axial Velocity Index"] = pd.cut(df["Axial Velocity Avg (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Min Horizontal Velocity Index"] = pd.cut(df["Horizontal Velocity Min (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Max Horizontal Velocity Index"] = pd.cut(df["Horizontal Velocity Max (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Avg Horizontal Velocity Index"] = pd.cut(df["Horizontal Velocity Avg (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Min Vertical Velocity Index"] = pd.cut(df["Vertical Velocity Min (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Max Vertical Velocity Index"] = pd.cut(df["Vertical Velocity Max (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Avg Vertical Velocity Index"] = pd.cut(df["Vertical Velocity Avg (mm/s)"], bins=[0,c,d,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Min Temperature Index"] = pd.cut(df["Temperature Min (°C)"], bins=[0,e,f,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Max Temperature Index"] = pd.cut(df["Temperature Max (°C)"], bins=[0,e,f,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Avg Temperature Index"] = pd.cut(df["Temperature Avg (°C)"], bins=[0,e,f,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Min Audio Index"] = pd.cut(df["Audio Min (dB)"], bins=[0,g,h,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Max Audio Index"] = pd.cut(df["Audio Max (dB)"], bins=[0,g,h,1000], labels=["Good","Alert","Danger"])
    VibrationDF["Avg Audio Index"] = pd.cut(df["Audio Avg (dB)"], bins=[0,g,h,1000], labels=["Good","Alert","Danger"])
    
    VibrationDF["From"]= pd.to_datetime(df["From"]) #Converting 'From' column to datetime
    VibrationDF["To"] = pd.to_datetime(df["To"]) #Converting 'To' column to datetime
    VibrationDF.dropna(inplace=True) #Remove null values
    global globalVib #Declaring as a Global Variable so as to be used outside of Function
    globalVib = VibrationDF.copy()

    y1 = Dict[parameter_dropdown][0] #Takes the first element from value of key(parameter_dropdown) in Dict - used for Line Chart
    y2 = Dict[parameter_dropdown][1] #Takes the second element from value of key(parameter_dropdown) in Dict - used for Pie Chart
    
    df2 = VibrationDF.loc[(VibrationDF['From'] >= start_date) & (VibrationDF['To'] <= end_date)]
    
    #Create Piechart for Summary of parameters
    piechart = px.pie(
                data_frame=df2,
                names=y2,
                title = 'Summary of Parameter for Selected Date Range',
                template = "plotly_dark",
                color_discrete_sequence=["green", "yellow","red"],
                color_discrete_map={
                "Good": "green",
                "Alert": "yellow",
                "Danger": "red"},
                hole = 0.3,
                    )
    piechart.update_traces(textposition='inside', textinfo='percent+label')
    
    #Create Line graph
    line = px.line(df2, x='From',y=y1,markers=True,template = "plotly_dark")
    line.update_layout(xaxis = {'title':'Date'}, 
                       title={'text':'Trend of selected parameter with Date Range'},autosize=True)
    line.update_yaxes(matches=None)
    
    #Defining the Background color ranges based on User input(Dropdown) - Graph background will have relevant threshold limits
    if parameter_dropdown=="MinAcceleration" or parameter_dropdown=="MaxAcceleration" or parameter_dropdown=="AvgAcceleration":
        
        line.add_hrect(y0=80, y1=a, line_width=0, fillcolor="green", opacity=0.4)
        line.add_hrect(y0=a, y1=b, line_width=0, fillcolor="yellow", opacity=0.4)
        line.add_hrect(y0=b, y1=125, line_width=0, fillcolor="red", opacity=0.4)
        
    elif parameter_dropdown=="MinHorizontal" or parameter_dropdown=="MaxHorizontal" or parameter_dropdown=="AvgHorizontal" or parameter_dropdown=="MinVertical" or parameter_dropdown=="MaxVertical" or parameter_dropdown=="AvgVertical" or parameter_dropdown=="MinAxial" or parameter_dropdown=="MaxAxial" or parameter_dropdown=="AvgAxial":
        line.add_hrect(y0=0.2, y1=c, line_width=0, fillcolor="green", opacity=0.4)
        line.add_hrect(y0=c, y1=d, line_width=0, fillcolor="yellow", opacity=0.4)
        line.add_hrect(y0=d, y1=1.6, line_width=0, fillcolor="red", opacity=0.4)
    
    elif parameter_dropdown=="MinTemperature" or parameter_dropdown=="MaxTemperature"or parameter_dropdown=="AvgTemperature":
        line.add_hrect(y0=15, y1=e, line_width=0, fillcolor="green", opacity=0.4)
        line.add_hrect(y0=e, y1=f, line_width=0, fillcolor="yellow", opacity=0.4)
        line.add_hrect(y0=f, y1=40, line_width=0, fillcolor="red", opacity=0.4)
    
    elif parameter_dropdown=="MinAudio" or parameter_dropdown=="MaxAudio" or parameter_dropdown=="AvgAudio":
        line.add_hrect(y0=50, y1=g, line_width=0, fillcolor="green", opacity=0.4)
        line.add_hrect(y0=g, y1=h, line_width=0, fillcolor="yellow", opacity=0.4)
        line.add_hrect(y0=h, y1=150, line_width=0, fillcolor="red", opacity=0.4)
      
    return(piechart,line)

#Layout for Prediction Tab
predictions_layout = html.Div([
    
    dbc.Row([
             dbc.Col(html.H4("Forecast of Vibration Parameter")),
             dbc.Col(html.H4("Fault Analysis and Prediction"))
           ]),
    dbc.Row([
             dbc.Col(dcc.Dropdown(
            id='parameter_prediction', 
            optionHeight = 25,searchable=True,placeholder = "Please Select", clearable = True,
            multi = False,style={"width": "80%","color": "#222A2A" },
            value="AvgAcceleration", options=[
                
                    {"label":"Avg Total Acceleration","value":"AvgAcceleration"},
                    {"label":"Avg Axial Velocity","value":"AvgAxial"},
                    {"label":"Avg Horizontal Velocity","value":"AvgHorizontal"},
                    {"label":"Avg Vertical Velocity","value":"AvgVertical"},
                    {"label":"Avg Temperature","value":"AvgTemperature"},
                    {"label":"Avg Audio","value":"AvgAudio"},
                
                 ])),
             dbc.Col(dcc.Textarea(
        id='fault',
        value="Please select the dropdown parameter to see Fault Prediction",
        style={'width': '110%', 'height': 100},className='mb-4' 
    ))
             ]),
    dbc.Row([
             dbc.Col(dcc.Graph(id='PredictGraph')),
             dbc.Col(html.Img(id="image"),style={'width': '20%', 'height': 80})

    ])
    
    
 ])
   
#Define Callback for Prediction 
@app.callback(
    [Output('PredictGraph', 'figure'), Output('image','src'),Output('fault','value')],
    [
     Input('parameter_prediction', 'value')]
)

#Defining function for Prediction tab
def predict_graph(parameter_prediction):

    dff = df #Copying df
    dff['From']=pd.to_datetime(dff['From'],infer_datetime_format=True)
    dff = dff.set_index(['From'])
    dff.dropna(inplace=True)
    
    #Use Arima Model with best AIC. The data is validated to be stationary with Dickey-fuller test 
    #Since Data is stationary, No data transformation is done
    
    model = ARIMA((dff[Dict[parameter_prediction][0]]),order=(1,1,0)) #ARIMA Model with order = (1,1,0) that resulted in lowest AIC
    
    #fit Model
    results_ARIMA = model.fit(disp=-1)
    
    #Predict the result with plot_predict for next 3 days(48 data points per day * 3)
    Predict = results_ARIMA.plot_predict(1,len(dff)+48*3,alpha=0.05,exog=None, dynamic=False, plot_insample=True) #Predict with start and end values

    fig, ax = plt.subplots()
    ax.set_xticklabels([])
    f = plotly.tools.mpl_to_plotly(Predict) #Convert matplotlib format to plotly to show in the dashboard
    f.update_layout(width=500,height=300,template = "plotly_dark",font_color='white',title="Prediction Graph for next 3 days")
    f.update_layout(legend=dict(
    
        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=12,
            color="white"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ))
    f.update_xaxes(title_text='Time',showticklabels=False) 
    f.update_yaxes(title_text = Dict[parameter_prediction][0])

    f.update_yaxes(matches=None)
    new = globalVib #Assign to the global variable
    new.dropna(inplace=True)
    
    #Specific Conditions and Validations to predict faults for each parameter.
    #Conditions are based on ISO Standards and vibration diagnosis and understanding 
    
    if parameter_prediction=="AvgAcceleration":
        g = len(new[new["Avg Acceleration Index"]=='Good'])
        b = len(new["Avg Acceleration Index"])
        a = len(new[new["Avg Acceleration Index"]=='Alert'])
        d = len(new[new["Avg Acceleration Index"]=='Danger'])
        if g/b >= 0.7:
            fault="The Asset has been operating {} % of the time in 'Good' condition. Continue to monitor the asset. Periodically observe Spectrum and TWF for signs of deterioration".format(round(g/b*100),1)
            src="https://thumbs.gfycat.com/PitifulGoodDingo-size_restricted.gif"
        else:
            fault = "The Asset has been operating {} % of the time in 'Alert' and {} % of the time in 'Danger' condition. High Acceleration points to a possible Bearing damage or Lubrication issues. Please check the Spectrum and TWF for further evidence and address immediately".format((round(a/b*100),1),(round(d/g*100),1))
            src="https://thumbs.gfycat.com/MistyUniqueImperialeagle-max-1mb.gif"
            
    elif parameter_prediction=="AvgAxial":
        g = len(new[new["Avg Axial Velocity Index"]=='Good'])
        b = len(new["Avg Axial Velocity Index"])
        a = len(new[new["Avg Axial Velocity Index"]=='Alert'])
        d = len(new[new["Avg Axial Velocity Index"]=='Danger'])
        if g/b >= 0.7:
            fault="The Asset has been operating {} % of the time in 'Good' condition. Continue to monitor the asset.Periodically observe Spectrum and TWF for signs of deterioration".format(round(g/b*100),1)
            src="https://thumbs.gfycat.com/PitifulGoodDingo-size_restricted.gif"
        else:
            fault = "The Asset has been operating {}% of the time in 'Alert' and {}% of the time in 'Danger' condition. High Axial Velocity points to a possible Misalignment. Please check the Spectrum and TWF for further evidence and address immediately".format((round(a/b*100),1),(round(d/g*100),1))
            src="https://i.ytimg.com/vi/0Me_-fR64e0/hq720.jpg?sqp=-oaymwEhCK4FEIIDSFryq4qpAxMIARUAAAAAGAElAADIQj0AgKJD&rs=AOn4CLD1UMWGifSDO9ZSBbpPNcqXfglWkw"
        
    elif parameter_prediction=="AvgHorizontal":
        g = len(new[new["Avg Horizontal Velocity Index"]=='Good'])
        b = len(new["Avg Horizontal Velocity Index"])
        a = len(new[new["Avg Horizontal Velocity Index"]=='Alert'])
        d = len(new[new["Avg Horizontal Velocity Index"]=='Danger'])
        if g/b >= 0.7:
            fault="The Asset has been operating {} % of the time in 'Good' condition. Continue to monitor the asset.Periodically observe Spectrum and TWF for signs of deterioration".format(round(g/b*100),1)
            src="https://thumbs.gfycat.com/PitifulGoodDingo-size_restricted.gif"
        else:
            fault = "The Asset has been operating {}% of the time in 'Alert' and {}% of the time in 'Danger' condition. High Horizontal Velocity points to a possible Unbalance. Please check the Spectrum and TWF for further evidence and address immediately".format((round(a/b*100),1),(round(d/g*100),1))
            src="https://thumbs.gfycat.com/AcclaimedDisgustingGermanshepherd-max-1mb.gif" 
    
    elif parameter_prediction=="AvgVertical":
        g = len(new[new["Avg Vertical Velocity Index"]=='Good'])
        b = len(new["Avg Vertical Velocity Index"])
        a = len(new[new["Avg Vertical Velocity Index"]=='Alert'])
        d = len(new[new["Avg Vertical Velocity Index"]=='Danger'])
        if g/b >= 0.7:
            fault="The Asset has been operating {} % of the time in 'Good' condition. Continue to monitor the asset.Periodically observe Spectrum and TWF for signs of deterioration".format(round(g/b*100),1)
            src="https://thumbs.gfycat.com/PitifulGoodDingo-size_restricted.gif"
        else:
            fault = "The Asset has been operating {}% of the time in 'Alert' and {}% of the time in 'Danger' condition. High Vertical Velocity points to a possible Unbalance. Please check the Spectrum and TWF for further evidence and address immediately".format((round(a/b*100),1),(round(d/g*100),1))
            src="https://thumbs.gfycat.com/MistyUniqueImperialeagle-max-1mb.gif" 
            
    elif parameter_prediction=="AvgTemperature":
        g = len(new[new["Avg Temperature Index"]=='Good'])
        b = len(new["Avg Temperature Index"])
        a = len(new[new["Avg Temperature Index"]=='Alert'])
        d = len(new[new["Avg Temperature Index"]=='Danger'])
        if g/b >= 0.7:
            fault="The Asset has been operating {} % of the time in 'Good' condition. Continue to monitor the asset. Periodically observe Spindle temperatures and heat map".format(round(g/b*100),1)
            src="https://thumbs.gfycat.com/PitifulGoodDingo-size_restricted.gif"
        else:
            fault = "The Asset has been operating {}% of the time in 'Alert' and {}% of the time in 'Danger' condition. High Temperature points to a possible lubrication issue. Please check the Spindle Lubrication for further evidence and address immediately".format((round(a/b*100),1),(round(d/g*100),1))
            src="https://thumbs.gfycat.com/MistyUniqueImperialeagle-max-1mb.gif"
    
  
    elif parameter_prediction=="AvgAudio":
        g = len(new[new["Avg Audio Index"]=='Good'])
        b = len(new["Avg Audio Index"])
        a = len(new[new["Avg Audio Index"]=='Alert'])
        d = len(new[new["Avg Audio Index"]=='Danger'])
        if g/b >= 0.7:
            fault="The Asset has been operating {} % of the time in 'Good' condition. Continue to monitor the asset".format(round(g/b*100),1)
            src="https://thumbs.gfycat.com/PitifulGoodDingo-size_restricted.gif"
        else:
            fault = "The Asset has been operating {}% of the time in 'Alert' and {}% of the time in 'Danger' condition. High Audio points to a possible Looseness. Please check for Harmonics in Spectrum for further evidence and address immediately".format((round(a/b*100),1),(round(d/g*100),1))
            src="https://www.nexusplus.sk/sites/default/files/images/Voln1.gif"           
            
    return f,src,fault


#Style Dictionaries for Tabs
tabs_styles = {
    'height': '35px'
}
tab_style = {

    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}
    
#creating layout for the tabs    
app_tabs = html.Div(
    [
        dbc.Tabs(
            [
                
                dbc.Tab(label="Analysis", tab_id="tab-analysis", labelClassName="text-success font-weight-bold", activeLabelClassName="text-danger",style=tab_style, active_tab_style=tab_selected_style),
                dbc.Tab(label="Predictions", tab_id="tab-predictions", labelClassName="text-success font-weight-bold", activeLabelClassName="text-danger",style=tab_style, active_tab_style=tab_selected_style),
            ],
            id="tabs",
            active_tab="tab-analysis",
        ),
    ], 
)

#creating master layout for the app
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Vibration Analysis Dashboard",
                            style={"textAlign": "center"},className = "bg-info text-white"), width=12)),
    
    dbc.Row(dbc.Col(app_tabs, width=12)),
    html.Div(id='content', children=[])

])

#Callback for app layout
@app.callback(
    Output("content", "children"),
    [Input("tabs", "active_tab")]
)

#Function to choose layout based on selection
def switch_tab(tab_chosen):
    
    if tab_chosen == "tab-analysis":
        
        return analysis_layout
    elif tab_chosen == "tab-predictions":
        return predictions_layout
    return html.P("Not Displayed")


# Run app and display external
app.run_server(mode='external',host="localhost",port=8053)

Dash app running on http://localhost:8053/


C:\Users\ashwa\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning:


statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)


C:\Users\ashwa\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. foreca